In [27]:
import pandas as pd
import cobra
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

# Load GEM
model = cobra.io.load_model('textbook')

# Get stoichiometric matrix from GEM
S = cobra.util.array.create_stoichiometric_matrix(model, array_type='DataFrame')

In [28]:
# Get metabolites and reactions list from GEM

METABOLITES = pd.DataFrame()
METABOLITES['rdf:label'] = [m.id for m in model.metabolites]

REACTIONS = pd.DataFrame()
REACTIONS['rdf:label'] = [r.id for r in model.reactions]
REACTIONS['rdf:lower_bound'] = [r.lower_bound for r in model.reactions]
REACTIONS['rdf:upper_bound'] = [r.upper_bound for r in model.reactions]

In [29]:
# Get pairs of reaction/metabolite
PAIRS = pd.DataFrame()

for col in S.columns:
    for row in S.index:
        if S[col].loc[row]!=0: 
            pair = str(col) + "?" + str(row)
            has_metab = row
            has_reac = col

            stoic = 999
            pair_row = {"Name": pair, "has_metab": has_metab, "has_reac": has_reac, "has_stoichiometry":stoic}
                        
            PAIRS = PAIRS.append(pair_row, ignore_index=True)

for row in range(PAIRS.shape[0]):
    m = PAIRS['has_metab'].iloc[row]
    r = PAIRS['has_reac'].iloc[row]
    
    met_in_r = model.reactions.get_by_id(r).metabolites

    for met in met_in_r.keys():
        if str(met) == str(m): 
            PAIRS.loc[row, 'has_stoichiometry'] = met_in_r[met]

In [30]:
# save to csv
METABOLITES.to_csv('metabolites.csv')
REACTIONS.to_csv('reactions.csv')
PAIRS.to_csv('pairs.csv')